In [ ]:
import pandas as pd

# Importando o arquivo CSV que contém os dados. O separador usado é o ponto e vírgula (';').
df = pd.read_csv('analisecovid.csv', sep=';')

# Exibindo a quantidade de valores nulos (faltantes) em cada coluna do DataFrame.
display(df.isna().sum())

# Substituindo valores nulos nas colunas específicas ('IDA_ESTABELECIMENTO_SAUDE', 'FICOU_INTERNADO' e 'FOI_INTUBADO')
# pela string "não informado". Isso evita problemas durante as análises.
df[["IDA_ESTABELECIMENTO_SAUDE", "FICOU_INTERNADO", "FOI_INTUBADO"]] = df[["IDA_ESTABELECIMENTO_SAUDE", "FICOU_INTERNADO", "FOI_INTUBADO"]].fillna("não informado")

# Exibindo novamente a quantidade de valores nulos para verificar se as substituições foram feitas corretamente.
display(df.isna().sum())

# Criando a coluna 'POSSIVEL_COVID' como indicador de possíveis casos de COVID.
# Um caso é considerado "possível COVID" se a pessoa apresentou todos os sintomas especificados (febre, tosse, dor no peito, perda de cheiro ou sabor).
df['POSSIVEL_COVID'] = (
    (df['TEVE_FEBRE'] == 'Sim') &
    (df['TEVE_TOSSE'] == 'Sim') &
    (df['DOR_PEITO'] == 'Sim') &
    (df['PERDA_CHEIRO_SABOR'] == 'Sim')
)

# Convertendo os valores True/False da coluna 'POSSIVEL_COVID' para os valores mais intuitivos: 'sim' e 'não'.
df['POSSIVEL_COVID'] = df['POSSIVEL_COVID'].replace({True: 'sim', False: 'não'})

# Agrupando os dados por UF (Unidade Federativa) e 'POSSIVEL_COVID', calculando a frequência de cada combinação.
uf_pcovid = df.groupby(['UF', 'POSSIVEL_COVID']).size().unstack(fill_value=0)

# Agrupando os dados por SEXO e 'POSSIVEL_COVID', calculando a frequência de cada combinação.
sexo_pcovid = df.groupby(['SEXO', 'POSSIVEL_COVID']).size().unstack(fill_value=0)

# Definindo faixas etárias para categorizar as idades e criando uma nova coluna 'Faixa Etária'.
bins = [0, 18, 35, 60, 100]  # Intervalos de idade
labels = ['0-18', '19-35', '36-60', 'Acima de 60']  # Rótulos correspondentes
df['Faixa Etária'] = pd.cut(df['IDADE'], bins=bins, labels=labels, right=False)

# Agrupando os dados por faixa etária e 'POSSIVEL_COVID', calculando a frequência de cada combinação.
faixa_etaria_pcovid = df.groupby(['Faixa Etária', 'POSSIVEL_COVID']).size().unstack(fill_value=0)

# Agrupando os dados por ESCOLARIDADE e 'POSSIVEL_COVID', calculando a frequência de cada combinação.
escolaridade_pcovid = df.groupby(['ESCOLARIDADE', 'POSSIVEL_COVID']).size().unstack(fill_value=0)

# Para colunas de sintomas, calculamos as frequências de cada resposta.
sintomas_colunas = {
    'Fez Teste': df.groupby('FEZ_TESTE').size(),
    'Ficou Internado': df.groupby('FICOU_INTERNADO').size(),
    'Foi Intubado': df.groupby('FOI_INTUBADO').size(),
    'Ida ao Estabelecimento de Saúde': df.groupby('IDA_ESTABELECIMENTO_SAUDE').size()
}


# Exportando os resultados das análises para arquivos CSV, que serão usados para visualizações no Power BI.
uf_pcovid.to_csv('uf_pcovid.csv')
sexo_pcovid.to_csv('sexo_pcovid.csv')
faixa_etaria_pcovid.to_csv('faixa_etaria_pcovid.csv')
escolaridade_pcovid.to_csv('escolaridade_pcovid.csv')

# Exportando as colunas de sintomas como arquivos CSV separados.
for coluna, resultados in sintomas_colunas.items():
    resultados.to_csv(f'{coluna}_sim_nao.csv')



In [ ]:
# Filtrar apenas os casos onde a coluna "POSSIVEL_COVID" é igual a "sim".
# Isso permite focar exclusivamente nos casos considerados como possíveis COVID.
df_possivel_covid = df[df["POSSIVEL_COVID"] == "sim"]

# Agrupar os dados filtrados por Unidade Federativa (UF) e pela variável "FEZ_TESTE".
# Aqui calculamos a contagem de respostas para cada categoria de "FEZ_TESTE" (Sim/Não) em cada estado.
fez_teste_por_uf = df_possivel_covid.groupby("UF")["FEZ_TESTE"].value_counts(normalize=False).unstack(fill_value=0)

# Criar uma coluna "Total", que soma todas as respostas (Sim e Não) por estado.
# Isso ajuda a entender o número total de respostas analisadas em cada estado.
fez_teste_por_uf["Total"] = fez_teste_por_uf.sum(axis=1)

# Criar uma coluna "Sim (%)", que calcula a porcentagem de respostas "Sim" em relação ao total.
# Essa métrica oferece uma visão relativa da realização de testes por estado.
fez_teste_por_uf["Sim (%)"] = (fez_teste_por_uf["Sim"] / fez_teste_por_uf["Total"]) * 100

# Ordenar os estados pela quantidade de respostas "Sim" em ordem decrescente.
# Selecionar os 5 estados com mais testes realizados (Top 5).
top_5_teste = fez_teste_por_uf.sort_values("Sim", ascending=False).head(5)

# Ordenar os estados pela quantidade de respostas "Sim" em ordem crescente.
# Selecionar os 5 estados com menos testes realizados (Bottom 5).
menor_5_teste = fez_teste_por_uf.sort_values("Sim", ascending=True).head(5)

# Exportar os resultados das análises para arquivos CSV, separando os 5 estados com mais e menos testes realizados.
top_5_teste.to_csv("top_5_fez_teste.csv")
menor_5_teste.to_csv("menor_5_fez_teste.csv")

# Exibir os 5 estados com mais testes realizados no terminal para conferência.
display("Top 5 estados com mais testes realizados:")
display(top_5_teste)

# Exibir os 5 estados com menos testes realizados no terminal para conferência.
display("\nTop 5 estados com menos testes realizados:")
display(menor_5_teste)
